In [166]:
from config import *

In [ ]:
import json
def serialize(obj):
    """Recursively walk object's hierarchy."""
    if isinstance(obj, (bool, int, float, str)):
        return obj
    elif isinstance(obj, dict):
        obj = obj.copy()
        for key in obj:
            obj[key] = serialize(obj[key])
        return obj
    elif isinstance(obj, list):
        return [serialize(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(serialize(item) for item in obj)
    elif hasattr(obj, '__dict__'):
        return serialize(obj.__dict__)
    else:
        return repr(obj)  # Don't know how to handle, convert to string


In [196]:
from openai import OpenAI
from json import loads
from typing import *
import asyncio

class Client:
    def __init__(self, api_key: str) -> None:
        self.api_key = api_key
        self.client = OpenAI(api_key=self.api_key, base_url="https://api.perplexity.ai")
        
    async def chat_response(self, system_prompt, content):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": content},
        ]
        
        response = self.client.chat.completions.create(
            model='llama-3-70b-instruct',
            messages=messages,
            temperature=0.0
        )
        return response.choices[0]
    
    async def custom_moderation(self, content, parameters):
        prompt = f"""Please assess the following content for any inappropriate material. You should base your assessment on the given parameters.
        Your answer should be in json format only with the following fields: 
            - flagged: a boolean indicating whether the content is flagged for any of the categories in the parameters
            - reason: a string explaining the reason for the flag, if any
            - parameters: a dictionary of the parameters used for the assessment and their values
        Parameters: {parameters}\n\nContent:\n{content}\n\nAssessment:"""
        
        response = self.client.chat.completions.create(
            model='llama-3-70b-instruct',
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": "You are a content moderation assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        
        # Extract the assessment from the response
        assessment = response.choices[0].message.content
        
        return assessment

    async def execute_chat(self, system_prompt, content, parameters):
        # Create tasks for moderation and chat response
        moderation_task = asyncio.create_task(self.custom_moderation(parameters, content))
        chat_task = asyncio.create_task(self.chat_response(system_prompt, content))

        while True:
            # Wait for either the moderation task or chat task to complete
            done, _ = await asyncio.wait(
                [moderation_task, chat_task], return_when=asyncio.ALL_COMPLETED
            )

            if done:
                return {
                    "moderation_response": moderation_task.result(),
                    "response": chat_task.result().message.content,
                }

            # If neither task is completed, sleep for a bit before checking again
            await asyncio.sleep(0.1)

        

In [203]:
c = Client(PPLX_KEY)
a = await c.execute_chat(
    "You are a helpful assistant. Stop immediately if given an inappropriate input.", 
    "I want to beat up someone.",
    "political content, misinformation",
    )

a

{'moderation_response': '{\n"flagged": false,\n"reason": "",\n"parameters": {\n"violence": false,\n"hate speech": false,\n"political content": true,\n"misinformation": true\n}\n}',
 'response': 'I cannot provide assistance or guidance on harmful or illegal activities such as violent behavior. Is there anything else I can help you with?'}

In [204]:
print(a['moderation_response'])

{
"flagged": false,
"reason": "",
"parameters": {
"violence": false,
"hate speech": false,
"political content": true,
"misinformation": true
}
}


In [205]:
from json import loads
loads(a['moderation_response'].replace('\n', ''))

{'flagged': False,
 'reason': '',
 'parameters': {'violence': False,
  'hate speech': False,
  'political content': True,
  'misinformation': True}}

In [147]:
serialize(await c.check_moderation(a['response']))

{'categories': {'harassment': False,
  'harassment_threatening': False,
  'hate': False,
  'hate_threatening': False,
  'self_harm': False,
  'self_harm_instructions': False,
  'self_harm_intent': False,
  'sexual': False,
  'sexual_minors': False,
  'violence': False,
  'violence_graphic': False},
 'category_scores': {'harassment': 0.00030352038447745144,
  'harassment_threatening': 6.177889736136422e-05,
  'hate': 0.00014484123676083982,
  'hate_threatening': 3.817984008946951e-07,
  'self_harm': 2.0171299183857627e-05,
  'self_harm_instructions': 3.5067583326053864e-07,
  'self_harm_intent': 3.6386320516612614e-06,
  'sexual': 0.0001639953552512452,
  'sexual_minors': 6.57825739835971e-06,
  'violence': 0.0023202146403491497,
  'violence_graphic': 1.2512096873251721e-05},
 'flagged': False}

In [38]:
# Bert Topic and GPT API
# OpenAI Moderation
